In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
%matplotlib inline

sys.path.append('../modules')
# import ar_funcs
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
def get_filename_GEFSv12_reforecast(F):
    ## this is the number of hours of lead time
    ## for getting the correct filename
    
    # F = int((np.timedelta64(ndays, 'D')+np.timedelta64(nhours, 'h'))/np.timedelta64(1, 'h'))
    ## the F-lead that the files are saved as
    li = [3, 24, 27, 48, 51, 72, 75, 96, 99, 120, 123, 144, 147, 168, 171, 192, 195, 216, 219, 240]
    
    for idx in range(len(li)-1):
        if li[idx] < F <= li[idx+1]:
            pos1, pos2 = idx, idx+1

    return li[pos1], li[pos2]

In [4]:
yr = 2000
F = 72
mon = 1
day = 1

center_date = '{0}-{1}-{2}'.format(yr, mon, day)
center_date = pd.to_datetime(center_date)
end_date = center_date + timedelta(days=45)

In [5]:
%%time


## for each year between 2000 and 2019
date_lst = []
for i, yr in enumerate(range(2000, 2020)):
    ## get 45 days before date
    center_date = '{0}-{1}-{2}'.format(yr, mon, day)
    center_date = pd.to_datetime(center_date)
    start_date = center_date - timedelta(days=45)
    
    ## get 45 days after center date
    end_date = center_date + timedelta(days=45)

    ## make a list of dates between start_date and end_date
    dates = pd.date_range(start_date, end_date, freq='1D')
    
    date_lst.append(dates)
    
## concatenate all years together into single list    
final_lst = np.concatenate(date_lst)
## remove dates outside GEFSv12 Reforecast
idx = (pd.DatetimeIndex(final_lst).year < 2020) & (pd.DatetimeIndex(final_lst).year >= 2000)
final_lst = final_lst[idx]

print(start_date, end_date)

## load all days from the new subset
## create list of fnames
fname_lst = []
varname = 'prec' ## ivt, uv1000, 'freezing_level' or 'prec'

scratch_path = '/expanse/lustre/scratch/dnash/temp_project/mclimate/{0}/'.format(varname)
# scratch_path = path_to_data + 'preprocessed/GEFSv12_reforecast/{0}/'.format(varname)

## append filenames to a list
print('Gathering filenames ...')
for i, dt in enumerate(final_lst):
    ts = pd.to_datetime(str(dt)) 
    d = ts.strftime("%Y%m%d")

    if varname == 'prec':
        fname = scratch_path + '{1}_{0}.nc'.format(varname, d)
    else:
        F1, F2 = get_filename_GEFSv12_reforecast(F)
        fname = scratch_path + '{1}_{0}_F{2}_F{3}.nc'.format(varname, d, F1, F2)
    fname_lst.append(fname)

2018-11-17 00:00:00 2019-02-15 00:00:00
Gathering filenames ...
CPU times: user 595 ms, sys: 1.15 ms, total: 596 ms
Wall time: 617 ms


In [6]:
len(fname_lst[:2])

2

In [7]:
ds_lst = []
for i, fname in enumerate(fname_lst[:2]):
    ds = xr.open_dataset(fname)
    ds_lst.append(ds)

ds = xr.concat(ds_lst, dim='valid_time')


ValueError: valid_time already exists as coordinate or variable name.

In [10]:
idx = np.timedelta64(int(np.timedelta64(F, 'h')/np.timedelta64(1, 'ns')), 'ns')
def preprocess_freezing_level(ds):
    ds = ds.sel(step=idx) # select the specific lead step   
    return ds
    
ds = xr.open_mfdataset(fname_lst, concat_dim="valid_time", 
                       combine="nested", engine='netcdf4', chunks={"lat": 100, "lon": 100},
                       preprocess=preprocess_freezing_level)
ds

<xarray.Dataset>
Dimensions:     (latitude: 281, number: 5, longitude: 479, valid_time: 1775)
Coordinates:
    step        timedelta64[ns] 3 days
  * latitude    (latitude) float64 70.0 69.75 69.5 69.25 ... 0.75 0.5 0.25 0.0
  * number      (number) int64 0 1 2 3 4
  * longitude   (longitude) float64 -179.5 -179.2 -179.0 ... -60.5 -60.25 -60.0
    time        (valid_time) datetime64[ns] 2000-01-01 2000-01-02 ... 2019-02-15
    surface     float64 0.0
  * valid_time  (valid_time) datetime64[ns] 2000-01-04 2000-01-05 ... 2019-02-18
Data variables:
    tp          (valid_time, number, latitude, longitude) float32 dask.array<chunksize=(1, 5, 281, 479), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-06-13T13:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
### Read the dataset
print('Reading the data ...')
### Select the xx hr lead time step
## this is the index value for selecting the timestep in the dataset
idx = np.timedelta64(int(np.timedelta64(F, 'h')/np.timedelta64(1, 'ns')), 'ns')

def preprocess(ds):
    ds = ds.drop_vars(["ivtu", "ivtv"])
    ds = ds.sel(step=idx) # select the 24 hr lead step
    
    return ds

## use xr.open_mfdataset to read all the files within that ssn clim
ds = xr.open_mfdataset(fname_lst, concat_dim="valid_time", combine="nested", engine='netcdf4', chunks={"lat": 100, "lon": 100}, preprocess=preprocess)

print('Calculating quantiles...')
## need to rechunk so time is a single chunk
ds = ds.chunk(dict(valid_time=-1))

# Percentile will be a set range of percentiles including <90th, then every 0.1 until 100th/MAX
# I might add 75th-90th, and < 75th
a = np.array([0, .75, .9])
b = np.arange(.91, 1.001, 0.01)
quantile_arr = np.concatenate((a, b), axis=0)

## Calculate the percentiles
ivt_mclimate = ds.quantile(quantile_arr, dim=['valid_time', 'number'], skipna=True)

## add dayofyear and lead to coordinates
ivt_mclimate = ivt_mclimate.assign_coords(step=F)
ivt_mclimate = ivt_mclimate.expand_dims('step')
period = pd.Period("2023-{0}-{1}".format(mon, day), freq='H')
ivt_mclimate = ivt_mclimate.assign_coords(dayofyear=period.day_of_year)
ivt_mclimate = ivt_mclimate.expand_dims('dayofyear')
ivt_mclimate

# write to netCDF
fname = os.path.join(path_to_data, 'preprocessed/GEFSv12_reforecast_mclimate_ivt_{0}{1}_{2}hr-lead.nc'.format(mon, day, F))
ivt_mclimate.load().to_netcdf(path=fname, mode = 'w', format='NETCDF4')